In [1]:
#!pip install fpdf

In [2]:
import requests,pandas as pd
import json
from pandas import json_normalize

import sklearn
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams

import os
import shutil
import numpy as np
import calendar
from datetime import datetime
from fpdf import FPDF


rcParams['axes.spines.top'] = False
rcParams['axes.spines.right'] = False
pd.set_option("display.max_columns", 32)

In [3]:
Year = '2014'
Make = 'HONDA'
Model = 'ACCORD'

In [4]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('JMACData05.xlsx', engine='xlsxwriter')

Ratings: NHTSA's New Car Assessment Program rates vehicles to determine crashworthiness and rollover safety. The safety ratings are gathered during controlled crash and rollover tests conducted at NHTSA research facilities. Vehicles with a rating of five stars indicate the highest safety rating, whereas a one star indicates the lowest rating. NCAP provides consumers with important information in making a new car purchase, and for comparable vehicle models, it provides a direct comparison of the crashworthiness of vehicles. Information from this program also exists on the required labels on each new vehicle for sale.

In [5]:
data_url = f'https://api.nhtsa.gov/SafetyRatings/modelyear/{Year}/make/{Make}/model/{Model}'
response=requests.get(data_url)

dict = json.loads(response.text)
Get_VID = pd.json_normalize(dict['Results']) 
Get_VID = Get_VID.reset_index(drop=True)
VEHID = Get_VID['VehicleId'][0]

In [6]:
data_url = f'https://api.nhtsa.gov/SafetyRatings/VehicleId/{VEHID}'
response=requests.get(data_url)
print("Request to get the Safety Ratings for the required vehicle variant by passing in the VehicleId")

dict = json.loads(response.text)
VECHILEID_Query = pd.json_normalize(dict['Results']) 

VECHILEID_Query1 = VECHILEID_Query[['OverallRating', 'OverallFrontCrashRating', 'FrontCrashDriversideRating', 'FrontCrashPassengersideRating', 
                                    'OverallSideCrashRating', 'SideCrashDriversideRating', 'SideCrashPassengersideRating', 'combinedSideBarrierAndPoleRating-Front', 
                                    'combinedSideBarrierAndPoleRating-Rear', 'sideBarrierRating-Overall', 'RolloverRating', 'RolloverPossibility', 
                                   'ComplaintsCount', 'RecallsCount', 'InvestigationCount', 'ModelYear', 'Make', 'Model', 'VehicleId', 'VehicleDescription']]
VECHILEID_Query1 = VECHILEID_Query1.set_index('VehicleDescription')
VECHILEID_Query1['Vehicle'] = (Year + ' ' + Make + ' ' + Model)
VECHILEID_Query1.to_excel(writer, sheet_name='Sheet1-Safety')
VECHILEID_Query1


Request to get the Safety Ratings for the required vehicle variant by passing in the VehicleId


,OverallRating,OverallFrontCrashRating,FrontCrashDriversideRating,FrontCrashPassengersideRating,OverallSideCrashRating,SideCrashDriversideRating,SideCrashPassengersideRating,combinedSideBarrierAndPoleRating-Front,combinedSideBarrierAndPoleRating-Rear,sideBarrierRating-Overall,RolloverRating,RolloverPossibility,ComplaintsCount,RecallsCount,InvestigationCount,ModelYear,Make,Model,VehicleId,Vehicle
VehicleDescription,,,,,,,,,,,,,,,,,,,,
2014 Honda Accord 4 DR FWD,5,4,4,4,5,5,5,5,5,5,5,0.099,737,3,2,2014,HONDA,ACCORD,8211,2014 HONDA ACCORD


Recalls: Manufacturers who determine that a product or piece of original equipment either has a safety defect, or is not in compliance with federal safety standards, are required to notify NHTSA within five business days. NHTSA requires that manufacturers file a defect and noncompliance report as well as quarterly recall status reports, in compliance with Federal Regulation 49 (the National Traffic and Motor Safety Act) Part 573, which identifies the requirements for safety recalls. NHTSA stores this information and the data can be used to search for recall information related to specific NHTSA campaigns and product types.

In [7]:
data_url = f'https://api.nhtsa.gov/recalls/recallsByVehicle?make={Make}&model={Model}&modelYear={Year}'
print("Request to get the Manufacturer Recalls for the requested vehicle")
response=requests.get(data_url)
dict = json.loads(response.text)
RecallV = pd.json_normalize(dict['results']) 
RecallV['Vehicle'] = (Year + ' ' + Make + ' ' + Model)
RecallV.to_excel(writer, sheet_name='Sheet2A-RecallDetail')
RecallV 

Request to get the Manufacturer Recalls for the requested vehicle


,Manufacturer,NHTSACampaignNumber,parkIt,parkOutSide,ReportReceivedDate,Component,Summary,Consequence,Remedy,Notes,ModelYear,Make,Model,Vehicle
0,Honda (American Honda Motor Co.),17V418000,False,False,29/06/2017,ELECTRICAL SYSTEM:12V/24V/48V BATTERY,Honda (American Honda Motor Co.) is recalling ...,An electrical short increases the risk of a fire.,"Honda will notify owners, and dealers will rep...",Owners may also contact the National Highway T...,2014,HONDA,ACCORD,2014 HONDA ACCORD
1,Honda (American Honda Motor Co.),15V121000,False,False,02/03/2015,ENGINE AND ENGINE COOLING:ENGINE,American Honda Motor Co. (Honda) is recalling ...,Loss of engine power may result in a vehicle s...,"Honda will notify owners, and dealers will rep...",Owners may also contact the National Highway T...,2014,HONDA,ACCORD,2014 HONDA ACCORD
2,Honda (American Honda Motor Co.),20V769000,False,False,10/12/2020,POWER TRAIN:DRIVELINE:DRIVESHAFT,Honda (American Honda Motor Co.) is recalling ...,A broken drive shaft may cause a sudden loss o...,"Honda will notify owners, and dealers will ins...",Owners may also contact the National Highway T...,2014,HONDA,ACCORD,2014 HONDA ACCORD


In [8]:
#RecallType = RecallV .groupby('Component').size().sort_values(ascending=False)
#RecallType.to_excel(writer, sheet_name='Sheet2B-RecallSummary')

#RecallType

Complaints: Complaint information entered into NHTSA’s Office of Defects Investigation vehicle owner's complaint database is used with other data sources to identify safety issues that warrant investigation and to determine if a safety-related defect trend exists. Complaint information is also analyzed to monitor existing recalls for proper scope and adequacy.

In [9]:
data_url = f'https://api.nhtsa.gov/complaints/complaintsByVehicle?make={Make}&model={Model}&modelYear={Year}'
print("Request to get the List of Filed Complaints for the requested vehicle")
response=requests.get(data_url)
dict = json.loads(response.text)
ComplainV = pd.json_normalize(dict['results']) 
ComplainV['Vehicle'] = (Year + ' ' + Make + ' ' + Model)
ComplainV.to_excel(writer, sheet_name='Sheet3A-ComplainDetail')
ComplainV 

Request to get the List of Filed Complaints for the requested vehicle


,odiNumber,manufacturer,crash,fire,numberOfInjuries,numberOfDeaths,dateOfIncident,dateComplaintFiled,vin,components,summary,products,Vehicle
0,11461857,Honda (American Honda Motor Co.),False,False,0,0,03/25/2022,04/23/2022,1HGCR2F38EA,POWER TRAIN,My son was driving and the car just shut down....,"[{'type': 'Vehicle', 'productYear': '2014', 'p...",2014 HONDA ACCORD
1,11461279,Honda (American Honda Motor Co.),True,False,1,0,05/11/2017,04/19/2022,1HGCR2F32EA,STEERING,"LOSS OF STEERING CONTROL! 11 May 2017, drivin...","[{'type': 'Vehicle', 'productYear': '2014', 'p...",2014 HONDA ACCORD
2,11460902,Honda (American Honda Motor Co.),False,False,0,0,04/04/2022,04/15/2022,1HGCR6F78EA,FORWARD COLLISION AVOIDANCE,Automatic Breaking System activated upon appro...,"[{'type': 'Vehicle', 'productYear': '2014', 'p...",2014 HONDA ACCORD
3,11460925,Honda (American Honda Motor Co.),False,False,0,0,04/15/2022,04/15/2022,1HGCR2F39EA,STEERING,I am experiencing electric power steering loss...,"[{'type': 'Vehicle', 'productYear': '2014', 'p...",2014 HONDA ACCORD
4,11460928,Honda (American Honda Motor Co.),False,False,0,0,02/15/2022,04/15/2022,1HGCR2F34EA,ELECTRICAL SYSTEM,"When trying to start the vehicle, sometimes th...","[{'type': 'Vehicle', 'productYear': '2014', 'p...",2014 HONDA ACCORD
...,...,...,...,...,...,...,...,...,...,...,...,...,...
732,10561043,Honda (American Honda Motor Co.),False,False,0,0,01/21/2014,01/22/2014,1HGCR2F35EA,VISIBILITY,IT WAS A COLD DAY WITH WIND CHILLED DOWN TO SI...,"[{'type': 'Vehicle', 'productYear': '2014', 'p...",2014 HONDA ACCORD
733,10561089,Honda (American Honda Motor Co.),False,False,0,0,01/04/2014,01/22/2014,1HGCR2F79EA,SEATS,PURCHASED A NEW 2014 HONDA ACCORD EX FROM THE ...,"[{'type': 'Vehicle', 'productYear': '2014', 'p...",2014 HONDA ACCORD
734,10554291,Honda (American Honda Motor Co.),False,False,0,0,11/16/2013,12/01/2013,,SEATS,GOT THE CAR BRAND NEW FROM THE DEALERSHIP AFTE...,"[{'type': 'Vehicle', 'productYear': '2014', 'p...",2014 HONDA ACCORD
735,10545418,Honda (American Honda Motor Co.),False,False,0,0,09/21/2013,09/25/2013,,SEATS,HEADREST IS NOT ADJUSTABLE BACKWARDS OR FORWAR...,"[{'type': 'Vehicle', 'productYear': '2014', 'p...",2014 HONDA ACCORD


In [10]:
#ComplaintType = ComplainV.groupby('components').size().sort_values(ascending=False)

#ComplaintType.to_excel(writer, sheet_name='Sheet3B-ComplainSummary')
#ComplaintType

In [11]:
# Close the Pandas Excel writer and output the Excel file.
writer.save()